In [39]:
import csv
import ast

from tqdm import tqdm

### CSV Functions

In [40]:
def load_csv_data(file_path, bool_params):
    # Initialize an empty list to store the data
    data_list = []

    # Open the CSV file for reading
    with open(file_path, newline='', encoding="utf-8") as csvfile:
        # Create a CSV reader object
        csv_reader = csv.DictReader(csvfile)
        
        # Iterate through each row in the CSV file
        for row in csv_reader:
            # Append the row (as a dictionary) to the data_list
            row["choices"] = ast.literal_eval(row["choices"])

            for param in bool_params:
                if row[param].lower() == "true":
                    row[param] = True
                elif row[param].lower() == "false":
                    row[param] = False
                else:
                    raise TypeError(f"{param} data cannot be recognized")

            data_list.append(row)
    
    return data_list

def load_all_rephrase_data(split, dir_path, file_name):
    data = {}
    
    for s in split:
        file_path = f"{dir_path}/{s}{file_name}"
        data[s] = load_csv_data(file_path, [])
        # data[s] = load_csv_data(file_path, ["concept", "name", "option"])
    
    return data

def save_data(samples, file_path):
    # Get the keys from the first dictionary
    header = samples[0].keys()

    # Write the data to the CSV file
    with open(file_path, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for row in samples:
            writer.writerow(row)

    print(f'CSV file "{file_path}" has been created with the data.')

### Filter Functions

In [41]:
from sentence_transformers import SentenceTransformer, util
minilm_model = SentenceTransformer('all-MiniLM-L12-v2')
multilingual_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def compute_similarity(text1, text2, multilingual=False):
    if multilingual:
        model = multilingual_model
    else:
        model = minilm_model

    embeddings1 = model.encode([text1], convert_to_tensor=True)
    embeddings2 = model.encode([text2], convert_to_tensor=True)

    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    return float(cosine_scores[0][0])

In [42]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from better_profanity import profanity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

en_stemmer = PorterStemmer()
id_stemmer =  StemmerFactory().create_stemmer()

def get_input_text(item):
    return " ".join([item['question'], item['question_concept']] + item['choices']['text'])

def filter_concept(text, lang="english"):
    # Step 1: Lowercase both question and question_concept
    question = text["question"].lower()
    question_concept = text["question_concept"].lower()
    
    # Step 2: Check if question_concept appears in question
    if question_concept in question:
        return True

    # Step 3: If not, split and remove stopwords
    if lang != "sundanese":
        stop_words = stopwords.words(lang)
    else:
        stop_words = []

    concept_words = question_concept.split()
    concept_words = [word for word in concept_words if word not in stop_words]

    # Check if any of the remaining words in question_concept appear in question
    if any(word in question for word in concept_words):
        return True

    # Step 4: Stem words and check if any stem word appears in question
    
    if lang != "sundanese":
        if lang == "english":
            stemmer = en_stemmer
        elif lang == "indonesian":
            stemmer = id_stemmer
        
        question_stemmed = " ".join(stemmer.stem(word) for word in question.split())
        if any(word in question_stemmed for word in [stemmer.stem(w) for w in concept_words]):
            return True

    # Step 5: If none of the above conditions met, return False
    return False

def filter_profanity(text):
    all_texts = get_input_text(text)
    
    return not profanity.contains_profanity(all_texts)

In [43]:
split = ["validation", "test", "train"]
en_data = load_all_rephrase_data(split, "92123", "_rephrased_name_92123.csv")
id_data = load_all_rephrase_data(split, "translated_data/id", ".csv")
su_data = load_all_rephrase_data(split, "translated_data/su", ".csv")

id_en_data = load_all_rephrase_data(split, "backtranslation/id_en", ".csv")
su_en_data = load_all_rephrase_data(split, "backtranslation/su_en", ".csv")
su_id_data = load_all_rephrase_data(split, "backtranslation/su_id", ".csv")

In [45]:
id_en_threshold = 0.9
su_en_threshold = 0.85
su_id_threshold = 0.9

for s in split:
    print(f"Filtering {s} split")

    id_count = 0
    su_count = 0
    en_count = 0
    prof_count = 0

    id_en_count = 0
    su_en_count = 0
    su_id_count = 0

    for idx, item in tqdm(enumerate(en_data[s])):
        id_en = compute_similarity(get_input_text(item), get_input_text(id_en_data[s][idx]))
        su_en = compute_similarity(get_input_text(item), get_input_text(su_en_data[s][idx]))
        su_id = compute_similarity(get_input_text(id_data[s][idx]), get_input_text(su_id_data[s][idx]), multilingual=True)
        
        id_ca = filter_concept(id_data[s][idx], lang="indonesian")
        su_ca = filter_concept(su_data[s][idx], lang="sundanese")

        id_data[s][idx]["id_en_similarity"] = id_en
        id_data[s][idx]["id_en_decision"] = bool(id_en >= id_en_threshold)
        id_data[s][idx]["concept_appearance"] = id_ca

        su_data[s][idx]["su_en_similarity"] = su_en
        su_data[s][idx]["su_en_decision"] = bool(su_en >= su_en_threshold)
        su_data[s][idx]["su_id_similarity"] = su_id
        su_data[s][idx]["su_id_decision"] = bool(su_id >= su_id_threshold)
        su_data[s][idx]["concept_appearance"] = su_ca

        item["id_en_similarity"] = id_en
        item["id_en_decision"] = bool(id_en >= id_en_threshold)
        item["id_concept_appearance"] = id_ca
        item["su_en_similarity"] = su_en
        item["su_en_decision"] = bool(su_en >= su_en_threshold)
        item["su_id_similarity"] = su_id
        item["su_id_decision"] = bool(su_id >= su_id_threshold)
        item["su_concept_appearance"] = su_ca
        item["concept_appearance"] = filter_concept(item)
        item["not_contain_profanity"] = filter_profanity(item)
        
        if not item["concept_appearance"]:
            en_count += 1
        if not item["id_concept_appearance"]:
            id_count += 1
        if not item["su_concept_appearance"]:
            su_count += 1
        if not item["not_contain_profanity"]:
            prof_count += 1
        
        if id_en < id_en_threshold:
            id_en_count += 1
        if su_en < su_en_threshold:
            su_en_count += 1
        if su_id < su_id_threshold:
            su_id_count += 1
        
    print(f"EN Filtered Concept: {en_count}")
    print(f"ID Filtered Concept: {id_count}")
    print(f"SU Filtered Concept: {su_count}")
    print(f"ID-EN Filtered Threshold {id_en_threshold}: {id_en_count}")
    print(f"SU-EN Filtered Threshold {su_en_threshold}: {su_en_count}")
    print(f"SU-ID Filtered Threshold {su_id_threshold}: {su_id_count}")
    print(f"Filtered Profanity: {prof_count}")

    save_data(en_data[s], f"./filtered_data/en/{s}.csv")
    save_data(id_data[s], f"./filtered_data/id/{s}.csv")
    save_data(su_data[s], f"./filtered_data/su/{s}.csv")
    print()

Filtering validation split


274it [01:01,  4.47it/s]


EN Filtered Concept: 14
ID Filtered Concept: 58
SU Filtered Concept: 100
ID-EN Filtered Threshold 0.9: 63
SU-EN Filtered Threshold 0.85: 71
SU-ID Filtered Threshold 0.9: 44
Filtered Profanity: 14
CSV file "./filtered_data/en/validation.csv" has been created with the data.
CSV file "./filtered_data/id/validation.csv" has been created with the data.
CSV file "./filtered_data/su/validation.csv" has been created with the data.

Filtering test split


236it [00:51,  4.58it/s]


EN Filtered Concept: 17
ID Filtered Concept: 62
SU Filtered Concept: 98
ID-EN Filtered Threshold 0.9: 51
SU-EN Filtered Threshold 0.85: 60
SU-ID Filtered Threshold 0.9: 30
Filtered Profanity: 10
CSV file "./filtered_data/en/test.csv" has been created with the data.
CSV file "./filtered_data/id/test.csv" has been created with the data.
CSV file "./filtered_data/su/test.csv" has been created with the data.

Filtering train split


2162it [07:48,  4.62it/s]

EN Filtered Concept: 104
ID Filtered Concept: 449
SU Filtered Concept: 791
ID-EN Filtered Threshold 0.9: 511
SU-EN Filtered Threshold 0.85: 509
SU-ID Filtered Threshold 0.9: 288
Filtered Profanity: 86
CSV file "./filtered_data/en/train.csv" has been created with the data.
CSV file "./filtered_data/id/train.csv" has been created with the data.
CSV file "./filtered_data/su/train.csv" has been created with the data.



In [47]:
for s in split:
    en_filtered = []
    id_filtered = []
    su_filtered = []
    for idx, item in tqdm(enumerate(en_data[s])):
        if item["id_en_decision"] and item["su_en_decision"] and item["su_id_decision"] and item["concept_appearance"] and item["id_concept_appearance"] and item["su_concept_appearance"] and item["not_contain_profanity"]:
            en_filtered.append({
                'id': item['id'],
                'question': item['question'],
                'question_concept': item['question_concept'],
                'choices': {
                    'label': item['choices']['label'],
                    'text': item['choices']['text']
                },
                'answerKey': item['answerKey']
            })
            id_filtered.append({
                'id': id_data[s][idx]['id'],
                'question': id_data[s][idx]['question'],
                'question_concept': id_data[s][idx]['question_concept'],
                'choices': {
                    'label': id_data[s][idx]['choices']['label'],
                    'text': id_data[s][idx]['choices']['text']
                },
                'answerKey': id_data[s][idx]['answerKey']
            })
            su_filtered.append({
                'id': su_data[s][idx]['id'],
                'question': su_data[s][idx]['question'],
                'question_concept': su_data[s][idx]['question_concept'],
                'choices': {
                    'label': su_data[s][idx]['choices']['label'],
                    'text': su_data[s][idx]['choices']['text']
                },
                'answerKey': su_data[s][idx]['answerKey']
            })
    print(f"Count Filtered Data for {s}: {len(en_filtered)}")
    save_data(en_filtered, f"./filtered_data/en/{s}.csv")
    save_data(id_filtered, f"./filtered_data/id/{s}.csv")
    save_data(su_filtered, f"./filtered_data/su/{s}.csv")

274it [00:00, 136830.49it/s]


Count Filtered Data for validation: 101
CSV file "./filtered_data/en/validation.csv" has been created with the data.
CSV file "./filtered_data/id/validation.csv" has been created with the data.
CSV file "./filtered_data/su/validation.csv" has been created with the data.


236it [00:00, 236242.42it/s]


Count Filtered Data for test: 82
CSV file "./filtered_data/en/test.csv" has been created with the data.
CSV file "./filtered_data/id/test.csv" has been created with the data.
CSV file "./filtered_data/su/test.csv" has been created with the data.


2162it [00:00, 254207.37it/s]

Count Filtered Data for train: 840
CSV file "./filtered_data/en/train.csv" has been created with the data.
CSV file "./filtered_data/id/train.csv" has been created with the data.
CSV file "./filtered_data/su/train.csv" has been created with the data.
